# Kotlin 실습 미션
## 실전 프로젝트로 배우는 Kotlin

이 노트북에서는 지금까지 학습한 Kotlin 개념들을 활용하여 실제 애플리케이션을 만들어봅니다.

### 실습 프로젝트
1. 콘솔 Todo 앱
2. 계산기 앱
3. 가계부 앱

## 1. 콘솔 Todo 앱

할 일을 관리하는 간단한 콘솔 애플리케이션을 만들어봅니다.

In [ ]:
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

// Todo 데이터 모델
data class Todo(
    val id: Int,
    val title: String,
    var completed: Boolean = false,
    val createdAt: LocalDateTime = LocalDateTime.now(),
    var completedAt: LocalDateTime? = null
) {
    // 완료 상태 토글
    fun toggle() {
        completed = !completed
        completedAt = if (completed) LocalDateTime.now() else null
    }
    
    // 보기 좋은 출력 형식
    override fun toString(): String {
        val status = if (completed) "✓" else "○"
        val time = createdAt.format(DateTimeFormatter.ofPattern("MM-dd HH:mm"))
        return "[${status}] #${id} ${title} (생성: ${time})"
    }
}

// Todo 테스트
val todo = Todo(1, "Kotlin 학습하기")
println("생성된 Todo: $todo")

todo.toggle()
println("완료 처리 후: $todo")

In [ ]:
// Todo 관리자 클래스
class TodoManager {
    private val todos = mutableListOf<Todo>()
    private var nextId = 1
    
    // Todo 추가
    fun addTodo(title: String): Todo {
        val todo = Todo(nextId++, title)
        todos.add(todo)
        return todo
    }
    
    // 모든 Todo 조회
    fun getTodos(showCompleted: Boolean = true): List<Todo> {
        return if (showCompleted) {
            todos.toList()
        } else {
            todos.filter { !it.completed }
        }
    }
    
    // ID로 Todo 찾기
    fun findById(id: Int): Todo? = todos.find { it.id == id }
    
    // Todo 완료 상태 토글
    fun toggleTodo(id: Int): Boolean {
        val todo = findById(id)
        todo?.toggle()
        return todo != null
    }
    
    // Todo 삭제
    fun deleteTodo(id: Int): Boolean {
        return todos.removeIf { it.id == id }
    }
    
    // 통계 정보
    fun getStats(): Map<String, Int> {
        val total = todos.size
        val completed = todos.count { it.completed }
        val pending = total - completed
        
        return mapOf(
            "total" to total,
            "completed" to completed,
            "pending" to pending
        )
    }
    
    // 검색 기능
    fun searchTodos(keyword: String): List<Todo> {
        return todos.filter { 
            it.title.contains(keyword, ignoreCase = true) 
        }
    }
}

// TodoManager 테스트
val manager = TodoManager()

// Todo 추가
manager.addTodo("Kotlin 기초 문법 학습")
manager.addTodo("SpringBoot 프로젝트 생성")
manager.addTodo("REST API 구현")
manager.addTodo("데이터베이스 연동")
manager.addTodo("테스트 코드 작성")

println("=== 전체 할 일 목록 ===")
manager.getTodos().forEach { println(it) }

In [ ]:
// Todo 앱 사용 예제
println("\n=== Todo 완료 처리 ===")
manager.toggleTodo(1)
manager.toggleTodo(2)
println("1번, 2번 완료 처리됨")

println("\n=== 미완료 항목만 조회 ===")
manager.getTodos(showCompleted = false).forEach { println(it) }

println("\n=== 검색 기능 ===")
val searchResult = manager.searchTodos("API")
println("'API' 검색 결과:")
searchResult.forEach { println(it) }

println("\n=== 통계 정보 ===")
val stats = manager.getStats()
println("전체: ${stats["total"]}개")
println("완료: ${stats["completed"]}개")
println("대기: ${stats["pending"]}개")

// 완료율 계산
val completionRate = if (stats["total"]!! > 0) {
    (stats["completed"]!! * 100.0) / stats["total"]!!
} else 0.0
println("완료율: %.1f%%".format(completionRate))

In [ ]:
// Todo 앱 확장: 우선순위와 태그 기능
enum class Priority {
    LOW, MEDIUM, HIGH, URGENT
}

data class EnhancedTodo(
    val id: Int,
    val title: String,
    var completed: Boolean = false,
    val priority: Priority = Priority.MEDIUM,
    val tags: MutableSet<String> = mutableSetOf(),
    val createdAt: LocalDateTime = LocalDateTime.now()
) {
    fun addTag(tag: String) = tags.add(tag)
    fun removeTag(tag: String) = tags.remove(tag)
    
    override fun toString(): String {
        val status = if (completed) "✓" else "○"
        val priorityIcon = when (priority) {
            Priority.LOW -> "🟢"
            Priority.MEDIUM -> "🟡"
            Priority.HIGH -> "🟠"
            Priority.URGENT -> "🔴"
        }
        val tagStr = if (tags.isNotEmpty()) tags.joinToString(", ", " [Tags: ", "]") else ""
        return "[$status] $priorityIcon #$id $title$tagStr"
    }
}

// 향상된 Todo 테스트
val enhancedTodo = EnhancedTodo(
    id = 1,
    title = "긴급 버그 수정",
    priority = Priority.URGENT
)
enhancedTodo.addTag("bug")
enhancedTodo.addTag("production")

println("향상된 Todo: $enhancedTodo")

### 💡 학습 포인트
- **데이터 클래스**: Todo 정보를 간결하게 표현
- **클래스 설계**: 책임 분리 (Todo vs TodoManager)
- **컬렉션 함수**: filter, find, count 등 활용
- **확장성**: 우선순위, 태그 등 기능 추가 용이

## 2. 계산기 앱

다양한 연산을 지원하는 계산기를 만들어봅니다.

In [ ]:
// 연산 타입을 Sealed 클래스로 정의
sealed class Operation {
    data class Add(val a: Double, val b: Double) : Operation()
    data class Subtract(val a: Double, val b: Double) : Operation()
    data class Multiply(val a: Double, val b: Double) : Operation()
    data class Divide(val a: Double, val b: Double) : Operation()
    data class Power(val base: Double, val exponent: Double) : Operation()
    data class Sqrt(val number: Double) : Operation()
    data class Percentage(val value: Double, val total: Double) : Operation()
}

// 계산 결과
sealed class CalculationResult {
    data class Success(val value: Double) : CalculationResult()
    data class Error(val message: String) : CalculationResult()
}

// Operation 예시
val operations = listOf(
    Operation.Add(10.0, 5.0),
    Operation.Divide(10.0, 0.0),  // 에러 케이스
    Operation.Sqrt(-4.0)          // 에러 케이스
)

println("정의된 연산들:")
operations.forEach { println("  - $it") }

In [ ]:
// 계산기 클래스
class Calculator {
    private val history = mutableListOf<Pair<String, CalculationResult>>()
    
    // 계산 수행
    fun calculate(operation: Operation): CalculationResult {
        val result = when (operation) {
            is Operation.Add -> {
                CalculationResult.Success(operation.a + operation.b)
            }
            is Operation.Subtract -> {
                CalculationResult.Success(operation.a - operation.b)
            }
            is Operation.Multiply -> {
                CalculationResult.Success(operation.a * operation.b)
            }
            is Operation.Divide -> {
                if (operation.b == 0.0) {
                    CalculationResult.Error("0으로 나눌 수 없습니다")
                } else {
                    CalculationResult.Success(operation.a / operation.b)
                }
            }
            is Operation.Power -> {
                CalculationResult.Success(Math.pow(operation.base, operation.exponent))
            }
            is Operation.Sqrt -> {
                if (operation.number < 0) {
                    CalculationResult.Error("음수의 제곱근은 계산할 수 없습니다")
                } else {
                    CalculationResult.Success(Math.sqrt(operation.number))
                }
            }
            is Operation.Percentage -> {
                CalculationResult.Success((operation.value / operation.total) * 100)
            }
        }
        
        // 히스토리에 저장
        val expression = operationToString(operation)
        history.add(expression to result)
        
        return result
    }
    
    // 연산을 문자열로 변환
    private fun operationToString(operation: Operation): String {
        return when (operation) {
            is Operation.Add -> "${operation.a} + ${operation.b}"
            is Operation.Subtract -> "${operation.a} - ${operation.b}"
            is Operation.Multiply -> "${operation.a} × ${operation.b}"
            is Operation.Divide -> "${operation.a} ÷ ${operation.b}"
            is Operation.Power -> "${operation.base}^${operation.exponent}"
            is Operation.Sqrt -> "√${operation.number}"
            is Operation.Percentage -> "${operation.value}/${operation.total} × 100%"
        }
    }
    
    // 히스토리 조회
    fun getHistory() = history.toList()
    
    // 성공한 계산만 조회
    fun getSuccessfulCalculations() = history.filter { 
        it.second is CalculationResult.Success 
    }
    
    // 히스토리 초기화
    fun clearHistory() = history.clear()
}

// 계산기 테스트
val calc = Calculator()

// 다양한 연산 수행
val testOperations = listOf(
    Operation.Add(10.0, 5.0),
    Operation.Subtract(20.0, 8.0),
    Operation.Multiply(4.0, 7.0),
    Operation.Divide(100.0, 4.0),
    Operation.Divide(10.0, 0.0),  // 에러
    Operation.Power(2.0, 8.0),
    Operation.Sqrt(144.0),
    Operation.Sqrt(-4.0),         // 에러
    Operation.Percentage(75.0, 200.0)
)

println("=== 계산 수행 ===")
testOperations.forEach { op ->
    when (val result = calc.calculate(op)) {
        is CalculationResult.Success -> {
            println("✓ ${calc.getHistory().last().first} = ${result.value}")
        }
        is CalculationResult.Error -> {
            println("✗ ${calc.getHistory().last().first} → 에러: ${result.message}")
        }
    }
}

In [ ]:
// 계산기 확장: 메모리 기능
class ScientificCalculator : Calculator() {
    private var memory: Double = 0.0
    
    // 메모리 기능
    fun memoryStore(value: Double) {
        memory = value
        println("메모리에 저장: $value")
    }
    
    fun memoryRecall(): Double {
        println("메모리에서 호출: $memory")
        return memory
    }
    
    fun memoryClear() {
        memory = 0.0
        println("메모리 초기화")
    }
    
    fun memoryAdd(value: Double) {
        memory += value
        println("메모리에 더하기: $value (현재: $memory)")
    }
    
    // 연속 계산
    fun chain(initial: Double, operations: List<(Double) -> Operation>): CalculationResult {
        var current = initial
        
        for (op in operations) {
            when (val result = calculate(op(current))) {
                is CalculationResult.Success -> current = result.value
                is CalculationResult.Error -> return result
            }
        }
        
        return CalculationResult.Success(current)
    }
}

// 과학 계산기 테스트
val sciCalc = ScientificCalculator()

// 메모리 기능 테스트
println("\n=== 메모리 기능 테스트 ===")
sciCalc.calculate(Operation.Add(10.0, 20.0)).let {
    if (it is CalculationResult.Success) {
        sciCalc.memoryStore(it.value)
    }
}

val recalled = sciCalc.memoryRecall()
sciCalc.calculate(Operation.Multiply(recalled, 2.0))

// 연속 계산 테스트
println("\n=== 연속 계산 ===")
val chainResult = sciCalc.chain(10.0, listOf(
    { x -> Operation.Add(x, 5.0) },      // 10 + 5 = 15
    { x -> Operation.Multiply(x, 2.0) }, // 15 × 2 = 30
    { x -> Operation.Sqrt(x) }           // √30 ≈ 5.48
))

when (chainResult) {
    is CalculationResult.Success -> println("연속 계산 결과: ${chainResult.value}")
    is CalculationResult.Error -> println("연속 계산 에러: ${chainResult.message}")
}

In [ ]:
// 계산 히스토리 분석
println("\n=== 계산 히스토리 분석 ===")
val history = calc.getHistory()
val successCount = history.count { it.second is CalculationResult.Success }
val errorCount = history.count { it.second is CalculationResult.Error }

println("총 계산 횟수: ${history.size}")
println("성공: $successCount")
println("에러: $errorCount")

// 성공한 계산의 평균값
val successfulValues = history
    .mapNotNull { (_, result) -> 
        (result as? CalculationResult.Success)?.value 
    }

if (successfulValues.isNotEmpty()) {
    val average = successfulValues.average()
    val max = successfulValues.maxOrNull()
    val min = successfulValues.minOrNull()
    
    println("\n성공한 계산 결과 통계:")
    println("평균: %.2f".format(average))
    println("최대값: $max")
    println("최소값: $min")
}

### 💡 학습 포인트
- **Sealed 클래스**: 연산 타입을 안전하게 표현
- **에러 처리**: Result 패턴으로 성공/실패 처리
- **상속**: 기본 계산기를 확장하여 과학 계산기 구현
- **함수형 프로그래밍**: 연속 계산을 함수 체이닝으로 구현

## 3. 가계부 앱

수입과 지출을 관리하는 가계부 애플리케이션을 만들어봅니다.

In [ ]:
// 거래 타입
enum class TransactionType {
    INCOME, EXPENSE
}

// 카테고리
enum class Category {
    // 수입 카테고리
    SALARY, BONUS, INVESTMENT, OTHER_INCOME,
    // 지출 카테고리
    FOOD, TRANSPORT, SHOPPING, ENTERTAINMENT, BILLS, EDUCATION, HEALTH, OTHER_EXPENSE
}

// 거래 데이터 모델
data class Transaction(
    val id: Long,
    val type: TransactionType,
    val amount: Int,
    val category: Category,
    val description: String,
    val date: LocalDateTime = LocalDateTime.now(),
    val tags: Set<String> = emptySet()
) {
    // 월 추출
    val month: String
        get() = date.format(DateTimeFormatter.ofPattern("yyyy-MM"))
    
    // 포맷된 금액
    val formattedAmount: String
        get() = "%,d원".format(amount)
    
    override fun toString(): String {
        val typeIcon = if (type == TransactionType.INCOME) "💰" else "💸"
        val dateStr = date.format(DateTimeFormatter.ofPattern("MM-dd HH:mm"))
        return "$typeIcon [$dateStr] $description: $formattedAmount ($category)"
    }
}

// 예시 거래
val sampleTransaction = Transaction(
    id = 1,
    type = TransactionType.EXPENSE,
    amount = 15000,
    category = Category.FOOD,
    description = "점심 외식",
    tags = setOf("외식", "회식")
)

println(sampleTransaction)

In [ ]:
// 가계부 관리 클래스
class ExpenseTracker {
    private val transactions = mutableListOf<Transaction>()
    private var nextId = 1L
    
    // 거래 추가
    fun addTransaction(
        type: TransactionType,
        amount: Int,
        category: Category,
        description: String,
        tags: Set<String> = emptySet()
    ): Transaction {
        val transaction = Transaction(
            id = nextId++,
            type = type,
            amount = amount,
            category = category,
            description = description,
            tags = tags
        )
        transactions.add(transaction)
        return transaction
    }
    
    // 수입 추가 (편의 메서드)
    fun addIncome(amount: Int, category: Category, description: String) =
        addTransaction(TransactionType.INCOME, amount, category, description)
    
    // 지출 추가 (편의 메서드)
    fun addExpense(amount: Int, category: Category, description: String) =
        addTransaction(TransactionType.EXPENSE, amount, category, description)
    
    // 전체 거래 조회
    fun getAllTransactions() = transactions.toList()
    
    // 기간별 거래 조회
    fun getTransactionsByMonth(year: Int, month: Int): List<Transaction> {
        return transactions.filter {
            it.date.year == year && it.date.monthValue == month
        }
    }
    
    // 카테고리별 거래 조회
    fun getTransactionsByCategory(category: Category) = 
        transactions.filter { it.category == category }
    
    // 총 수입/지출 계산
    fun getTotalByType(type: TransactionType) = 
        transactions
            .filter { it.type == type }
            .sumOf { it.amount }
    
    // 잔액 계산
    fun getBalance(): Int {
        val income = getTotalByType(TransactionType.INCOME)
        val expense = getTotalByType(TransactionType.EXPENSE)
        return income - expense
    }
    
    // 카테고리별 통계
    fun getCategoryStatistics(type: TransactionType): Map<Category, Int> {
        return transactions
            .filter { it.type == type }
            .groupBy { it.category }
            .mapValues { (_, transactions) -> 
                transactions.sumOf { it.amount }
            }
    }
}

// 가계부 테스트
val tracker = ExpenseTracker()

// 수입 추가
tracker.addIncome(3000000, Category.SALARY, "월급")
tracker.addIncome(500000, Category.BONUS, "성과급")

// 지출 추가
tracker.addExpense(8500, Category.FOOD, "점심 식사")
tracker.addExpense(1250, Category.TRANSPORT, "지하철")
tracker.addExpense(4500, Category.FOOD, "커피")
tracker.addExpense(12000, Category.ENTERTAINMENT, "영화")
tracker.addExpense(35000, Category.SHOPPING, "옷 구매")
tracker.addExpense(55000, Category.BILLS, "통신비")
tracker.addExpense(25000, Category.FOOD, "저녁 외식")
tracker.addExpense(100000, Category.EDUCATION, "온라인 강의")

println("=== 전체 거래 내역 ===")
tracker.getAllTransactions().forEach { println(it) }

In [ ]:
// 월간 보고서 생성
class MonthlyReport(
    val year: Int,
    val month: Int,
    private val tracker: ExpenseTracker
) {
    private val monthlyTransactions = tracker.getTransactionsByMonth(year, month)
    
    fun generate(): String {
        val income = monthlyTransactions
            .filter { it.type == TransactionType.INCOME }
            .sumOf { it.amount }
        
        val expense = monthlyTransactions
            .filter { it.type == TransactionType.EXPENSE }
            .sumOf { it.amount }
        
        val balance = income - expense
        val savingsRate = if (income > 0) (balance * 100.0) / income else 0.0
        
        // 지출 카테고리별 통계
        val expenseByCategory = monthlyTransactions
            .filter { it.type == TransactionType.EXPENSE }
            .groupBy { it.category }
            .mapValues { (_, trans) -> trans.sumOf { it.amount } }
            .toList()
            .sortedByDescending { it.second }
        
        return buildString {
            appendLine("=== ${year}년 ${month}월 가계부 보고서 ===")
            appendLine()
            appendLine("📊 수입/지출 요약")
            appendLine("  총 수입: %,d원".format(income))
            appendLine("  총 지출: %,d원".format(expense))
            appendLine("  잔액: %,d원".format(balance))
            appendLine("  저축률: %.1f%%".format(savingsRate))
            appendLine()
            appendLine("💸 지출 카테고리 TOP 5")
            expenseByCategory.take(5).forEachIndexed { index, (category, amount) ->
                val percentage = if (expense > 0) (amount * 100.0) / expense else 0.0
                appendLine("  ${index + 1}. $category: %,d원 (%.1f%%)".format(amount, percentage))
            }
            appendLine()
            appendLine("📈 일일 평균")
            val daysInMonth = java.time.YearMonth.of(year, month).lengthOfMonth()
            appendLine("  평균 지출: %,d원/일".format(expense / daysInMonth))
        }
    }
}

// 월간 보고서 생성
val now = LocalDateTime.now()
val report = MonthlyReport(now.year, now.monthValue, tracker)
println("\n${report.generate()}")

In [ ]:
// 예산 관리 기능
class BudgetManager(
    private val tracker: ExpenseTracker
) {
    private val budgets = mutableMapOf<Category, Int>()
    
    // 예산 설정
    fun setBudget(category: Category, amount: Int) {
        budgets[category] = amount
        println("${category} 예산 설정: %,d원".format(amount))
    }
    
    // 예산 대비 사용률 확인
    fun checkBudgetStatus(year: Int, month: Int): Map<Category, BudgetStatus> {
        val monthlyExpenses = tracker.getTransactionsByMonth(year, month)
            .filter { it.type == TransactionType.EXPENSE }
            .groupBy { it.category }
            .mapValues { (_, trans) -> trans.sumOf { it.amount } }
        
        return budgets.mapValues { (category, budget) ->
            val spent = monthlyExpenses[category] ?: 0
            val percentage = if (budget > 0) (spent * 100.0) / budget else 0.0
            
            BudgetStatus(
                budget = budget,
                spent = spent,
                remaining = budget - spent,
                percentage = percentage
            )
        }
    }
    
    data class BudgetStatus(
        val budget: Int,
        val spent: Int,
        val remaining: Int,
        val percentage: Double
    ) {
        val status: String
            get() = when {
                percentage >= 100 -> "초과 ⚠️"
                percentage >= 80 -> "주의 🟡"
                percentage >= 50 -> "정상 🟢"
                else -> "여유 💚"
            }
    }
}

// 예산 관리 테스트
val budgetManager = BudgetManager(tracker)

// 예산 설정
budgetManager.setBudget(Category.FOOD, 300000)
budgetManager.setBudget(Category.TRANSPORT, 100000)
budgetManager.setBudget(Category.ENTERTAINMENT, 50000)
budgetManager.setBudget(Category.SHOPPING, 200000)

// 예산 상태 확인
println("\n=== 예산 사용 현황 ===")
val budgetStatus = budgetManager.checkBudgetStatus(now.year, now.monthValue)
budgetStatus.forEach { (category, status) ->
    println("\n$category ${status.status}")
    println("  예산: %,d원".format(status.budget))
    println("  사용: %,d원 (%.1f%%)".format(status.spent, status.percentage))
    println("  잔액: %,d원".format(status.remaining))
}

In [ ]:
// 지출 패턴 분석
class SpendingAnalyzer(private val tracker: ExpenseTracker) {
    
    // 요일별 평균 지출
    fun analyzeByDayOfWeek(): Map<String, Double> {
        val expenses = tracker.getAllTransactions()
            .filter { it.type == TransactionType.EXPENSE }
        
        val dayNames = listOf("월", "화", "수", "목", "금", "토", "일")
        
        return expenses
            .groupBy { it.date.dayOfWeek.value - 1 }
            .mapValues { (_, trans) -> 
                trans.map { it.amount }.average() 
            }
            .mapKeys { (dayIndex, _) -> dayNames[dayIndex] }
    }
    
    // 시간대별 지출 분포
    fun analyzeByHour(): Map<Int, Int> {
        return tracker.getAllTransactions()
            .filter { it.type == TransactionType.EXPENSE }
            .groupBy { it.date.hour }
            .mapValues { (_, trans) -> trans.size }
            .toSortedMap()
    }
    
    // 지출 증가 추세 분석
    fun analyzeTrend(category: Category, months: Int = 3): List<Pair<String, Int>> {
        val now = LocalDateTime.now()
        
        return (0 until months).map { monthsAgo ->
            val targetDate = now.minusMonths(monthsAgo.toLong())
            val monthStr = targetDate.format(DateTimeFormatter.ofPattern("yyyy-MM"))
            
            val amount = tracker.getTransactionsByMonth(targetDate.year, targetDate.monthValue)
                .filter { it.type == TransactionType.EXPENSE && it.category == category }
                .sumOf { it.amount }
            
            monthStr to amount
        }.reversed()
    }
}

// 분석 실행
val analyzer = SpendingAnalyzer(tracker)

println("\n=== 지출 패턴 분석 ===")

// 요일별 분석
println("\n📅 요일별 평균 지출")
val dayAnalysis = analyzer.analyzeByDayOfWeek()
dayAnalysis.forEach { (day, avg) ->
    println("  ${day}요일: %,.0f원".format(avg))
}

// 가장 지출이 많은 요일
val maxDay = dayAnalysis.maxByOrNull { it.value }
if (maxDay != null) {
    println("  → 가장 지출이 많은 요일: ${maxDay.key}요일")
}

### 💡 학습 포인트
- **Enum 클래스**: 카테고리와 거래 타입 정의
- **데이터 분석**: 그룹핑, 필터링, 집계 함수 활용
- **보고서 생성**: buildString으로 포맷된 문자열 생성
- **확장 가능한 설계**: 예산 관리, 패턴 분석 등 기능 추가

## 종합 실습 과제

지금까지 만든 앱들을 결합하여 종합적인 개인 생산성 도구를 만들어보세요.

In [ ]:
// 과제: 개인 생산성 대시보드
// TODO: 다음 기능들을 통합한 앱을 만들어보세요

// 1. Todo와 가계부 연동
//    - "통신비 납부" Todo 완료 시 자동으로 지출 기록
//    - 예산 초과 시 Todo 자동 생성 ("지출 줄이기")

// 2. 목표 관리 시스템
//    - 저축 목표 설정 및 진행률 추적
//    - Todo와 연동하여 목표 달성 태스크 관리

// 3. 리포트 자동화
//    - 주간/월간 종합 리포트 생성
//    - Todo 완료율, 지출 현황, 목표 달성률 포함

// 4. 알림 시스템
//    - 예산 초과 경고
//    - Todo 마감일 알림
//    - 목표 달성 축하 메시지

// 여기에 코드를 작성하세요
